<a href="https://colab.research.google.com/github/MaliheDahmardeh/Olympic-History/blob/main/Olympic-History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
import numpy as np
import pandas as pd
import os
import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC ,SVR
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pandas.core.algorithms import mode

In [110]:
!ls

athlete_events.csv  noc_regions.csv  sample_data


In [111]:
df_events = pd.read_csv('athlete_events.csv')

In [112]:
df_noc = pd.read_csv('noc_regions.csv')

In [113]:
df = pd.merge(df_events,df_noc,on='NOC',how='inner')

In [114]:
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,602,Abudoureheman,M,22.0,182.0,75.0,China,CHN,2000 Summer,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,NaN,China,NaN
3,1463,Ai Linuer,M,25.0,160.0,62.0,China,CHN,2004 Summer,2004,Summer,Athina,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",NaN,China,NaN
4,1464,Ai Yanhan,F,14.0,168.0,54.0,China,CHN,2016 Summer,2016,Summer,Rio de Janeiro,Swimming,Swimming Women's 200 metres Freestyle,NaN,China,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270762,120575,Mamorallo Tjoka,F,23.0,150.0,56.0,Lesotho,LES,2008 Summer,2008,Summer,Beijing,Athletics,Athletics Women's Marathon,NaN,Lesotho,NaN
270763,120575,Mamorallo Tjoka,F,27.0,150.0,56.0,Lesotho,LES,2012 Summer,2012,Summer,London,Athletics,Athletics Women's Marathon,NaN,Lesotho,NaN
270764,122166,M'apotlaki Ts'elho,F,15.0,NaN,NaN,Lesotho,LES,1996 Summer,1996,Summer,Atlanta,Athletics,Athletics Women's 4 x 100 metres Relay,NaN,Lesotho,NaN
270765,122215,Lefa Tsapi,M,23.0,170.0,63.0,Lesotho,LES,1984 Summer,1984,Summer,Los Angeles,Boxing,Boxing Men's Welterweight,NaN,Lesotho,NaN


In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270767 entries, 0 to 270766
Data columns (total 17 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      270767 non-null  int64  
 1   Name    270767 non-null  object 
 2   Sex     270767 non-null  object 
 3   Age     261305 non-null  float64
 4   Height  210684 non-null  float64
 5   Weight  207982 non-null  float64
 6   Team    270767 non-null  object 
 7   NOC     270767 non-null  object 
 8   Games   270767 non-null  object 
 9   Year    270767 non-null  int64  
 10  Season  270767 non-null  object 
 11  City    270767 non-null  object 
 12  Sport   270767 non-null  object 
 13  Event   270767 non-null  object 
 14  Medal   39774 non-null   object 
 15  region  270746 non-null  object 
 16  notes   5039 non-null    object 
dtypes: float64(3), int64(2), object(12)
memory usage: 37.2+ MB


In [116]:
df.shape

(270767, 17)

#Data Cleaning

In [117]:
df.drop(columns=['notes'],inplace=True)
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China
2,602,Abudoureheman,M,22.0,182.0,75.0,China,CHN,2000 Summer,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,NaN,China
3,1463,Ai Linuer,M,25.0,160.0,62.0,China,CHN,2004 Summer,2004,Summer,Athina,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",NaN,China
4,1464,Ai Yanhan,F,14.0,168.0,54.0,China,CHN,2016 Summer,2016,Summer,Rio de Janeiro,Swimming,Swimming Women's 200 metres Freestyle,NaN,China
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270762,120575,Mamorallo Tjoka,F,23.0,150.0,56.0,Lesotho,LES,2008 Summer,2008,Summer,Beijing,Athletics,Athletics Women's Marathon,NaN,Lesotho
270763,120575,Mamorallo Tjoka,F,27.0,150.0,56.0,Lesotho,LES,2012 Summer,2012,Summer,London,Athletics,Athletics Women's Marathon,NaN,Lesotho
270764,122166,M'apotlaki Ts'elho,F,15.0,NaN,NaN,Lesotho,LES,1996 Summer,1996,Summer,Atlanta,Athletics,Athletics Women's 4 x 100 metres Relay,NaN,Lesotho
270765,122215,Lefa Tsapi,M,23.0,170.0,63.0,Lesotho,LES,1984 Summer,1984,Summer,Los Angeles,Boxing,Boxing Men's Welterweight,NaN,Lesotho


In [118]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9462
Height     60083
Weight     62785
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     230993
region        21
dtype: int64

In [119]:
df.duplicated().sum()

1385

In [120]:
df.drop_duplicates(inplace=True)

In [121]:
df.duplicated().sum()

0

In [122]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9303
Height     58726
Weight     61437
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     229619
region        21
dtype: int64

Fill  null value for Age, Height and Weight with mean of them

Mean of height


In [123]:
mean_height=df["Height"].mean()
mean_height

175.34423420173172

In [124]:
df["Height"]=df["Height"].fillna(mean_height)
df["Height"]

0         180.000000
1         170.000000
2         182.000000
3         160.000000
4         168.000000
             ...    
270762    150.000000
270763    150.000000
270764    175.344234
270765    170.000000
270766    175.000000
Name: Height, Length: 269382, dtype: float64

Mean of weight

In [125]:
mean_weight=df["Weight"].mean()
mean_weight

70.70890860564091

In [126]:
df["Weight"]=df["Weight"].fillna(mean_weight)
df["Weight"]

0         80.000000
1         60.000000
2         75.000000
3         62.000000
4         54.000000
            ...    
270762    56.000000
270763    56.000000
270764    70.708909
270765    63.000000
270766    75.000000
Name: Weight, Length: 269382, dtype: float64

Mean of Age

In [127]:
mean_age=df["Age"].mean()
mean_age

25.45754174693074

In [128]:
df["Age"]=df["Age"].fillna(mean_age)
df["Age"]

0         24.0
1         23.0
2         22.0
3         25.0
4         14.0
          ... 
270762    23.0
270763    27.0
270764    15.0
270765    23.0
270766    20.0
Name: Age, Length: 269382, dtype: float64

In [129]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age            0
Height         0
Weight         0
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     229619
region        21
dtype: int64

In [130]:
df.describe

<bound method NDFrame.describe of             ID                Name Sex   Age      Height     Weight     Team  \
0            1           A Dijiang   M  24.0  180.000000  80.000000    China   
1            2            A Lamusi   M  23.0  170.000000  60.000000    China   
2          602       Abudoureheman   M  22.0  182.000000  75.000000    China   
3         1463           Ai Linuer   M  25.0  160.000000  62.000000    China   
4         1464           Ai Yanhan   F  14.0  168.000000  54.000000    China   
...        ...                 ...  ..   ...         ...        ...      ...   
270762  120575     Mamorallo Tjoka   F  23.0  150.000000  56.000000  Lesotho   
270763  120575     Mamorallo Tjoka   F  27.0  150.000000  56.000000  Lesotho   
270764  122166  M'apotlaki Ts'elho   F  15.0  175.344234  70.708909  Lesotho   
270765  122215          Lefa Tsapi   M  23.0  170.000000  63.000000  Lesotho   
270766  122299       Mosolesa Tsie   M  20.0  175.000000  75.000000  Lesotho   

     

Drop other null values

In [131]:
df.dropna(inplace=True)

In [132]:
df.isnull().sum()

ID        0
Name      0
Sex       0
Age       0
Height    0
Weight    0
Team      0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
region    0
dtype: int64

#Data